In [50]:
!pip3 install statsmodels

In [57]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [58]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = '{:,.2f}'.format

seed=42
np.random.seed = 42

In [41]:
#columns
usefullCollumns_list = ['id','usableAreas','totalAreas','parkingSpaces','address_zone'
,'address_neighborhood','suites','bedrooms','bathrooms','pricingInfos_price','pricingInfos_yearlyIptu'
,'pricingInfos_monthlyCondoFee','interestingFlag']

dtype_list = ['usableAreas','totalAreas','parkingSpaces','suites','bedrooms'
,'pricingInfos_price','pricingInfos_yearlyIptu','pricingInfos_monthlyCondoFee']

In [42]:
#reading
dataset = pd.read_csv('data/processed/train_cleaned.csv.gzip',compression='gzip', sep =';')
dataset[[x for x in dtype_list]] = dataset[[x for x in dtype_list]].astype('float')

/home/willy/Desktop/imoveis/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
_ = dataset.address_neighborhood.nunique()/dataset[dataset.usableAreas!=0].shape[0]
print(f'razão: # de bairros/# de linhas = {_:.2f}')

razão: # de bairros/# de linhas = 0.03


In [44]:
dataset['usableAreasFlag']=dataset['usableAreas'].apply(lambda x: 1 if x>0 else 0)